In [ ]:
%matplotlib ipympl
import numpy as np
import tqdm
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.ndimage import gaussian_filter
from scipy.spatial.distance import cdist

from vizlib import *

In [ ]:
filename = "/Volumes/DATA/amatter_24_01_2024_card_worms_only.xyzv"
filename = "/media/matt/DATA/amatter_24_01_2024_card_worms_only.xyzv"
#filename = "/media/matt/DATA/data_29_01_2024/amatter_29_01_2024_worms_only.xyzv"
N,nframes,pos_data,vel_data = load_data(filename)

In [ ]:
vector_field = get_particle_vecs_frame(N,80,-1,pos_data)

In [ ]:
"""plotting the vectors of each of the individual particles"""
fig,ax = plt.subplots(1,1,figsize=(10,10))
quiver = ax.quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax.set_box_aspect(1)
ax.set_adjustable("datalim")
plt.show()

In [ ]:
def local_average_director(data, radius):
    """
    Perform local averaging of the director data.

    Parameters:
    - data: (N, 6) array with columns for x-position, y-position, z-position, x-dir, y-dir, z-dir.
    - radius: Radius of the local neighborhood for averaging.

    Returns:
    - averaged_data: (N, 6) array with columns for x-position, y-position, z-position, averaged x-dir, averaged y-dir, averaged z-dir.
    """
    num_points = data.shape[0]
    averaged_data = np.zeros_like(data)
    empty_count = 0
    for i in range(num_points):
        # Calculate distances to all other points
        distances = cdist(data[:, :2], data[i, :2].reshape(1, -1))[0]

        # Identify points within the specified radius
        neighbors_indices = np.where(distances <= radius)[0]
        if neighbors_indices.size > 0:
            # Average the director components within the local neighborhood
            averaged_x_dir = np.mean(data[neighbors_indices, 3])
            averaged_y_dir = np.mean(data[neighbors_indices, 4])

            # Update the averaged_data array
            averaged_data[i, :] = [data[i, 0], data[i, 1], 0.0,averaged_x_dir, averaged_y_dir,0.0]
        else:
            averaged_data[i, :] = [data[i, 0], data[i, 1], 0.0, data[i,3], data[i,4], 0.0]
            empty_count += 1
    print("Points with no neighbors (increase radius?):",empty_count,"/",num_points,"\t",(empty_count/num_points)*100,"%")
    return averaged_data

radius = 100.0  # You can adjust the radius based on your simulation requirements
averaged_data = local_average_director(vector_field, radius)


In [ ]:
"""plotting the vectors of each of the averaged particle data"""
fig,(ax1, ax2) = plt.subplots(1, 2,figsize=(12,6))
quiver = ax1.quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax1.set_box_aspect(1)
ax1.set_adjustable("datalim")
ax1.set_title('Filament Vectors')
quiver = ax2.quiver(averaged_data[:,0], #x
                   averaged_data[:,1], #y
                   averaged_data[:,3], #u
                   averaged_data[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax2.set_box_aspect(1)
ax2.set_title('Locally-Averaged Directors')
ax2.set_adjustable("datalim")
plt.show()


In [ ]:
max_x = np.max(vector_field[:,0])
min_x = np.min(vector_field[:,0])
max_y = np.max(vector_field[:,1])
min_y = np.min(vector_field[:,1])
print("X:",min_x,max_x,max_x-min_x)
print("Y:",min_y,max_y,max_y-min_y)

# create structured grid
interp_step = 1
grid_x,grid_y = np.meshgrid(np.arange(min_x,max_x,interp_step),
                           np.arange(min_y,max_y,interp_step))
num_interpolation_points = 100
grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                           np.linspace(min_y,max_y,num=num_interpolation_points))
# Interpolate the x and y components separately using bilinear interpolation
grid_u = interpolate.griddata((vector_field[:,0], vector_field[:,1]), vector_field[:,3], (grid_x, grid_y), method='linear')
grid_v = interpolate.griddata((vector_field[:,0], vector_field[:,1]), vector_field[:,4], (grid_x, grid_y), method='linear')
# merged everything back together to make single object
structured_vector_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))
# normalize the vector parts
n = np.sqrt(structured_vector_field[:,2] ** 2 + structured_vector_field[:,3] ** 2)
structured_vector_field[:,2] = structured_vector_field[:,2]/n
structured_vector_field[:,3] = structured_vector_field[:,3]/n

# replace nans with 0.0
np.nan_to_num(structured_vector_field[:,2], copy=False, nan=0.0)
np.nan_to_num(structured_vector_field[:,3], copy=False, nan=0.0)

"""plotting the vectors of each of the averaged particle data"""
fig,axs = plt.subplots(2, 2,figsize=(14,14))
fig.suptitle("Scipy GridData Interpolation")
quiver = axs[0,0].quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,0].set_box_aspect(1)
axs[0,0].set_adjustable("datalim")
axs[0,0].set_title('Filament Vectors')
quiver = axs[0,1].quiver(structured_vector_field[:,0], #x
                   structured_vector_field[:,1], #y
                   structured_vector_field[:,2], #u
                   structured_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,1].set_box_aspect(1)
axs[0,1].set_title('Interpolated Directors')
axs[0,1].set_adjustable("datalim")

quiver = axs[1,0].quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,0].set_box_aspect(1)
axs[1,0].set_adjustable("datalim")
axs[1,0].set_xlim(100,200)
axs[1,0].set_ylim(100,200)
quiver = axs[1,1].quiver(structured_vector_field[:,0], #x
                   structured_vector_field[:,1], #y
                   structured_vector_field[:,2], #u
                   structured_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,1].set_box_aspect(1)
axs[1,1].set_adjustable("datalim")
axs[1,1].set_xlim(100,200)
axs[1,1].set_ylim(100,200)
plt.show()

In [ ]:
max_x = np.max(vector_field[:,0])
min_x = np.min(vector_field[:,0])
max_y = np.max(vector_field[:,1])
min_y = np.min(vector_field[:,1])
print("X:",min_x,max_x,max_x-min_x)
print("Y:",min_y,max_y,max_y-min_y)

# create structured grid
interp_step = 1
grid_x,grid_y = np.meshgrid(np.arange(min_x,max_x,interp_step),
                           np.arange(min_y,max_y,interp_step))
num_interpolation_points = 100
grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                           np.linspace(min_y,max_y,num=num_interpolation_points))
print(vector_field[:,0].shape)
print(np.random.uniform(0,10,100).shape)
# Interpolate the x and y components separately
rbf_x = interpolate.RBFInterpolator(np.column_stack((vector_field[:,0],vector_field[:,1])),vector_field[:,3])
grid_u = rbf_x(np.column_stack((grid_x.ravel(),grid_y.ravel())))

rbf_y = interpolate.RBFInterpolator(np.column_stack((vector_field[:,0],vector_field[:,1])),vector_field[:,4])
grid_v = rbf_y(np.column_stack((grid_x.ravel(),grid_y.ravel())))

# merged everything back together to make single object
rbf_vector_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))
# normalize the vector parts
n = np.sqrt(rbf_vector_field[:,2] ** 2 + rbf_vector_field[:,3] ** 2)
rbf_vector_field[:,2] = rbf_vector_field[:,2]/n
rbf_vector_field[:,3] = rbf_vector_field[:,3]/n
"""plotting the vectors of each of the averaged particle data"""
fig,(ax1, ax2) = plt.subplots(1, 2,figsize=(12,6))
fig.suptitle("Scipy RBF Interpolation")
quiver = ax1.quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax1.set_box_aspect(1)
ax1.set_adjustable("datalim")
ax1.set_title('Filament Vectors')
quiver = ax2.quiver(rbf_vector_field[:,0], #x
                   rbf_vector_field[:,1], #y
                   rbf_vector_field[:,2], #u
                   rbf_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax2.set_box_aspect(1)
ax2.set_title('Interpolated Directors')
ax2.set_adjustable("datalim")
plt.show()

In [ ]:
max_x = np.max(vector_field[:,0])
min_x = np.min(vector_field[:,0])
max_y = np.max(vector_field[:,1])
min_y = np.min(vector_field[:,1])
print("X:",min_x,max_x,max_x-min_x)
print("Y:",min_y,max_y,max_y-min_y)

# create structured grid
interp_step = 1
grid_x,grid_y = np.meshgrid(np.arange(min_x,max_x,interp_step),
                           np.arange(min_y,max_y,interp_step))
num_interpolation_points = 150
grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                           np.linspace(min_y,max_y,num=num_interpolation_points))
print(vector_field[:,0].shape)
print(np.random.uniform(0,10,100).shape)
# Interpolate the x and y components separately
nnd_x = interpolate.NearestNDInterpolator(np.column_stack((vector_field[:,0],vector_field[:,1])),vector_field[:,3])
grid_u = nnd_x(np.column_stack((grid_x.ravel(),grid_y.ravel())))

nnd_y = interpolate.NearestNDInterpolator(np.column_stack((vector_field[:,0],vector_field[:,1])),vector_field[:,4])
grid_v = nnd_y(np.column_stack((grid_x.ravel(),grid_y.ravel())))

# merged everything back together to make single object
nnd_vector_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))
# normalize the vector parts
n = np.sqrt(nnd_vector_field[:,2] ** 2 + nnd_vector_field[:,3] ** 2)
nnd_vector_field[:,2] = nnd_vector_field[:,2]/n
nnd_vector_field[:,3] = nnd_vector_field[:,3]/n
"""plotting the vectors of each of the averaged particle data"""
fig,axs = plt.subplots(2, 2,figsize=(14,14))
fig.suptitle("Scipy NearestND Interpolation")
quiver = axs[0,0].quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,0].set_box_aspect(1)
axs[0,0].set_adjustable("datalim")
axs[0,0].set_title('Filament Vectors')
quiver = axs[0,1].quiver(nnd_vector_field[:,0], #x
                   nnd_vector_field[:,1], #y
                   nnd_vector_field[:,2], #u
                   nnd_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,1].set_box_aspect(1)
axs[0,1].set_title('Interpolated Directors')
axs[0,1].set_adjustable("datalim")

quiver = axs[1,0].quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,0].set_box_aspect(1)
axs[1,0].set_adjustable("datalim")
axs[1,0].set_xlim(100,200)
axs[1,0].set_ylim(100,200)
quiver = axs[1,1].quiver(nnd_vector_field[:,0], #x
                   nnd_vector_field[:,1], #y
                   nnd_vector_field[:,2], #u
                   nnd_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,1].set_box_aspect(1)
axs[1,1].set_adjustable("datalim")
axs[1,1].set_xlim(100,200)
axs[1,1].set_ylim(100,200)
plt.show()

In [ ]:
max_x = np.max(vector_field[:,0])
min_x = np.min(vector_field[:,0])
max_y = np.max(vector_field[:,1])
min_y = np.min(vector_field[:,1])
print("X:",min_x,max_x,max_x-min_x)
print("Y:",min_y,max_y,max_y-min_y)

# create structured grid
interp_step = 1
grid_x,grid_y = np.meshgrid(np.arange(min_x,max_x,interp_step),
                           np.arange(min_y,max_y,interp_step))
num_interpolation_points = 150#75
grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                           np.linspace(min_y,max_y,num=num_interpolation_points))

# Interpolate the x and y components separately
lnd_x = interpolate.LinearNDInterpolator(np.column_stack((vector_field[:,0],vector_field[:,1])),vector_field[:,3])
grid_u = lnd_x(np.column_stack((grid_x.ravel(),grid_y.ravel())))

lnd_y = interpolate.LinearNDInterpolator(np.column_stack((vector_field[:,0],vector_field[:,1])),vector_field[:,4])
grid_v = lnd_y(np.column_stack((grid_x.ravel(),grid_y.ravel())))

# merged everything back together to make single object
lnd_vector_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))
# normalize the vector parts
n = np.sqrt(lnd_vector_field[:,2] ** 2 + lnd_vector_field[:,3] ** 2)
lnd_vector_field[:,2] = lnd_vector_field[:,2]/n
lnd_vector_field[:,3] = lnd_vector_field[:,3]/n
"""plotting the vectors of each of the averaged particle data"""
fig,axs = plt.subplots(2, 2,figsize=(14,14))
fig.suptitle("Scipy LinearND Interpolation")
quiver = axs[0,0].quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,0].set_box_aspect(1)
axs[0,0].set_adjustable("datalim")
axs[0,0].set_title('Filament Vectors')
quiver = axs[0,1].quiver(lnd_vector_field[:,0], #x
                   lnd_vector_field[:,1], #y
                   lnd_vector_field[:,2], #u
                   lnd_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,1].set_box_aspect(1)
axs[0,1].set_title('Interpolated Directors')
axs[0,1].set_adjustable("datalim")

quiver = axs[1,0].quiver(vector_field[:,0], #x
                   vector_field[:,1], #y
                   vector_field[:,3], #u
                   vector_field[:,4], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,0].set_box_aspect(1)
axs[1,0].set_adjustable("datalim")
axs[1,0].set_xlim(100,200)
axs[1,0].set_ylim(100,200)
quiver = axs[1,1].quiver(lnd_vector_field[:,0], #x
                   lnd_vector_field[:,1], #y
                   lnd_vector_field[:,2], #u
                   lnd_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,1].set_box_aspect(1)
axs[1,1].set_adjustable("datalim")
axs[1,1].set_xlim(100,200)
axs[1,1].set_ylim(100,200)
plt.show()

In [ ]:
grid_u = lnd_vector_field[:,2]
grid_v = lnd_vector_field[:,3]
q_tensor = np.zeros((grid_x.shape[0],grid_x.shape[1],2,2))
np.nan_to_num(grid_u, copy=False, nan=0.0)
np.nan_to_num(grid_v, copy=False, nan=0.0)
print(grid_x.shape,grid_u.shape,150*150)
grid_u = np.reshape(grid_u,grid_x.shape)
grid_v = np.reshape(grid_v,grid_x.shape)
for i in range(grid_x.shape[0]):
   for j in range(grid_x.shape[1]):
      S_i = grid_u[i,j]
      S_j = grid_v[i,j]

      q_tensor[i,j,0,0] = 1.5 *(S_i * S_i - 0.5)
      q_tensor[i,j,0,1] = 1.5 *(S_i * S_j)
      q_tensor[i,j,1,0] = 1.5 *(S_i * S_j)
      q_tensor[i,j,1,1] = 1.5 *(S_j * S_j - 0.5)

print(q_tensor[1,1])
print(grid_u[1,1])

In [ ]:
dx = grid_x[0,1] - grid_x[0,0]
dy = grid_y[1,0] - grid_y[0,0]

dq11_tensor_dx = np.zeros_like(grid_x)
dq12_tensor_dx = np.zeros_like(grid_x)
dq11_tensor_dy = np.zeros_like(grid_y)
dq12_tensor_dy = np.zeros_like(grid_y)

dq11_tensor_dx[:, 1:-1] = (q_tensor[:, 2:, 0, 0] - q_tensor[:, :-2, 0, 0]) / (2 * dx)
dq11_tensor_dx[:, 0] = (q_tensor[:, 1, 0, 0] - q_tensor[:, 0, 0, 0]) / dx
dq11_tensor_dx[:, -1] = (q_tensor[:, -1, 0,0] - q_tensor[:, -2, 0, 0]) / dx

dq12_tensor_dx[:, 1:-1] = (q_tensor[:, 2:, 0, 1] - q_tensor[:, :-2, 0, 1]) / (2 * dx)
dq12_tensor_dx[:, 0] = (q_tensor[:, 1, 0, 1] - q_tensor[:, 0, 0, 1]) / dx
dq12_tensor_dx[:, -1] = (q_tensor[:, -1, 0,1] - q_tensor[:, -2, 0, 1]) / dx

dq11_tensor_dy[1:-1, :] = (q_tensor[2:, :, 0, 0] - q_tensor[:-2, :, 0, 0]) / (2 * dy)
dq11_tensor_dy[0, :] = (q_tensor[1, :, 0, 0] - q_tensor[0, :, 0, 0]) / dy
dq11_tensor_dy[-1, :] = (q_tensor[-1, :, 0, 0] - q_tensor[-2, :, 0, 0]) / dy

dq12_tensor_dy[1:-1, :] = (q_tensor[2:, :, 0, 1] - q_tensor[:-2, :, 0, 1]) / (2 * dy)
dq12_tensor_dy[0, :] = (q_tensor[1, :, 0, 1] - q_tensor[0, :, 0, 1]) / dy
dq12_tensor_dy[-1, :] = (q_tensor[-1, :, 0, 1] - q_tensor[-2, :, 0, 1]) / dy

dij = dq11_tensor_dx * dq12_tensor_dy - dq11_tensor_dy*dq12_tensor_dx
print(dij.shape)

In [ ]:
fig,ax2 = plt.subplots(1,1,figsize=(10,10))
quiver = ax2.quiver(lnd_vector_field[:,0], #x
                   lnd_vector_field[:,1], #y
                   lnd_vector_field[:,2], #u
                   lnd_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
pcm = ax2.imshow(dij,cmap="spring",alpha=0.5,origin="lower",extent=(structured_vector_field[:,0][0],structured_vector_field[:,0][-1],structured_vector_field[:,1][0],structured_vector_field[:,1][-1]))
fig.colorbar(pcm, ax=ax2)
ax2.set_box_aspect(1)
ax2.set_title('Interpolated Directors')
ax2.set_adjustable("datalim")
plt.show()

In [ ]:
from sklearn.preprocessing import minmax_scale
fig,(ax2,ax1) = plt.subplots(1,2,figsize=(14,7))
quiver = ax2.quiver(lnd_vector_field[:,0], #x
                   lnd_vector_field[:,1], #y
                   lnd_vector_field[:,2], #u
                   lnd_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
dij_rescaled = np.reshape(minmax_scale(dij.ravel(), feature_range=(-1,1)),grid_x.shape)
pcm = ax2.imshow(dij_rescaled,cmap="hot",alpha=0.5,origin="lower",extent=(structured_vector_field[:,0][0],structured_vector_field[:,0][-1],structured_vector_field[:,1][0],structured_vector_field[:,1][-1]))
#fig.colorbar(pcm, ax=ax2)
ax2.set_box_aspect(1)
ax2.set_title("(dQ11/dx)*(dQ12/dy) - (dQ11/dy)*(dQ12/dx)")
ax2.set_adjustable("datalim")
quiver = ax1.quiver(lnd_vector_field[:,0], #x
                   lnd_vector_field[:,1], #y
                   lnd_vector_field[:,2], #u
                   lnd_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
dij_rescaled = np.reshape(minmax_scale(dij.ravel(), feature_range=(-1,1)),grid_x.shape)
pcm = ax1.imshow(dij_rescaled,cmap="hot",alpha=0.5,origin="lower",extent=(structured_vector_field[:,0][0],structured_vector_field[:,0][-1],structured_vector_field[:,1][0],structured_vector_field[:,1][-1]))
fig.colorbar(pcm, ax=ax1)
ax1.set_box_aspect(1)
#ax2.set_title("(dQ11/dx)*(dQ12/dy) - (dQ11/dy)*(dQ12/dx)")
ax1.set_xlim(100,200)
ax1.set_ylim(100,200)
ax2.set_adjustable("datalim")
plt.show()

In [ ]:
u = grid_u
v = grid_v
E = np.zeros((num_interpolation_points,num_interpolation_points))
for i in np.arange(num_interpolation_points):
   for j in np.arange(num_interpolation_points):
      jnab = j
      inab = i + 1
      if inab >= num_interpolation_points-1:
            inab = i
      dp = u[i,j]*u[inab,jnab] + v[i,j]*v[inab,jnab]
      E[i,j] += 1-(dp*dp)

      jnab = j
      inab = i - 1
      if inab <= 0:
            inab = i
      dp = u[i,j]*u[inab,jnab] + v[i,j]*v[inab,jnab]
      E[i,j] += 1-(dp*dp)

      inab = i
      jnab = j + 1
      if jnab >= num_interpolation_points-1:
            jnab = j
      dp = u[i,j]*u[inab,jnab] + v[i,j]*v[inab,jnab]
      E[i,j] += 1-(dp*dp)

      inab = i
      jnab = j - 1
      if jnab <= 0:
            jnab = j
      dp = u[i,j]*u[inab,jnab] + v[i,j]*v[inab,jnab]
      E[i,j] += 1-(dp*dp)
E_norm = (E-np.min(E))/(np.max(E)-np.min(E))
fig,ax2 = plt.subplots(1,1,figsize=(10,10))
quiver = ax2.quiver(lnd_vector_field[:,0], #x
                   lnd_vector_field[:,1], #y
                   lnd_vector_field[:,2], #u
                   lnd_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
pcm = ax2.imshow(E_norm,cmap="hot",interpolation="bilinear",alpha=0.5,origin="lower",extent=(structured_vector_field[:,0][0],structured_vector_field[:,0][-1],structured_vector_field[:,1][0],structured_vector_field[:,1][-1]))
fig.colorbar(pcm, ax=ax2)
ax2.set_box_aspect(1)
ax2.set_title('Interpolated Directors')
ax2.set_adjustable("datalim")
plt.show()

In [ ]:
from scipy.ndimage import label, generate_binary_structure, binary_erosion, binary_dilation

def identify_defects(director_x, director_y, threshold=0.8, neighborhood_size=3):
    """
    Identify +1/2 and -1/2 defects in a structured director field.

    Parameters:
    - director_x: 2D array representing the x-component of the director field.
    - director_y: 2D array representing the y-component of the director field.
    - threshold: Threshold value to identify defects based on the topological charge.
    - neighborhood_size: Size of the neighborhood for morphological operations.

    Returns:
    - defects: 2D array where +1/2 defects are labeled as 0.5, -1/2 defects as -0.5, and other regions as 0.
    """
    # Calculate the topological charge
    topological_charge = director_x * np.gradient(director_y, axis=0) - director_y * np.gradient(director_x, axis=1)

    # Threshold the topological charge to identify defect regions
    defect_regions = np.abs(topological_charge) > threshold

    # Label connected components
    labeled_defects, num_labels = label(defect_regions)

    # Initialize the defects array
    defects = np.zeros_like(labeled_defects, dtype=float)

    # Analyze each labeled region to determine the defect type
    for label_value in range(1, num_labels + 1):
        # Extract the region corresponding to the current label
        region_mask = labeled_defects == label_value

        # Check the sign of the total winding number
        total_winding_number = np.sum(topological_charge[region_mask])

        # Label the defect type based on the winding number
        if total_winding_number > 0:
            defects[labeled_defects == label_value] = 0.5  # +1/2 defect
        elif total_winding_number < 0:
            defects[labeled_defects == label_value] = -0.5  # -1/2 defect

    # Apply morphological operations to refine defect regions
    struct = generate_binary_structure(2, neighborhood_size)
    defects = binary_erosion(defects, structure=struct)
    defects = binary_dilation(defects, structure=struct)

    return defects,topological_charge

# Identify defects
defects,topo_charge = identify_defects(grid_u, grid_v)

fig,ax1 = plt.subplots(1,1,figsize=(10,10))
quiver = ax1.quiver(structured_vector_field[:,0], #x
                   structured_vector_field[:,1], #y
                   structured_vector_field[:,2], #u
                   structured_vector_field[:,3], #v
                   pivot='mid',
                   headlength=0,
                   headwidth=0,
                   headaxislength=0,
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
pcm = ax1.imshow(topo_charge,cmap="spring",interpolation="bilinear",alpha=0.5,origin="lower",extent=(structured_vector_field[:,0][0],structured_vector_field[:,0][-1],structured_vector_field[:,1][0],structured_vector_field[:,1][-1]))
fig.colorbar(pcm, ax=ax1)
ax1.set_box_aspect(1)
ax1.set_title('Topological Charge')
ax1.set_adjustable("datalim")
plt.show()

### Velocity Data (Vorticity)

In [ ]:
def get_particle_vel_frame(N,iframe,pos_data,vel_data,disp=True):
   """ calculating the vectors of each of the individual particles"""
   worm_length = 80
   vector_field = np.zeros((N,6))
   num_worms = int(N/worm_length)
   if disp:
      print("num_worms:",num_worms)

   worm_view = np.array_split(pos_data[iframe,:,:],num_worms)
   vel_view = np.array_split(vel_data[iframe,:,:],num_worms)
   count = 0
   for iw in range(num_worms):
      for ip in range(len(worm_view[iw])):
         vec = np.zeros(3)
         pos = np.zeros(3)
         pos[0] = worm_view[iw][ip][0]
         pos[1] = worm_view[iw][ip][1]
         pos[2] = worm_view[iw][ip][2]
         vec[0] = vel_view[iw][ip][0] #X
         vec[1] = vel_view[iw][ip][1] #Y
         vec[2] = vel_view[iw][ip][2] #Z
         vector_field[count,0:3] = pos
         vector_field[count,3:6] = vec
         count += 1
         #print(ip,worm_view[iw][ip],vec)
   return vector_field
vel_field = get_particle_vel_frame(N,-1,pos_data,vel_data)

In [ ]:
"""plotting the vectors of each of the individual particles"""
fig,ax = plt.subplots(1,1,figsize=(10,10))
quiver = ax.quiver(vel_field[:,0], #x
                   vel_field[:,1], #y
                   vel_field[:,3], #u
                   vel_field[:,4], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
ax.set_box_aspect(1)
ax.set_adjustable("datalim")
plt.show()

In [ ]:
max_x = np.max(vel_field[:,0])
min_x = np.min(vel_field[:,0])
max_y = np.max(vel_field[:,1])
min_y = np.min(vel_field[:,1])
print("X:",min_x,max_x,max_x-min_x)
print("Y:",min_y,max_y,max_y-min_y)

# create structured grid
interp_step = 1
grid_x,grid_y = np.meshgrid(np.arange(min_x,max_x,interp_step),
                           np.arange(min_y,max_y,interp_step))
num_interpolation_points = 75
grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                           np.linspace(min_y,max_y,num=num_interpolation_points))

# Interpolate the x and y components separately
lnd_x = interpolate.LinearNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,3])
grid_u = lnd_x(np.column_stack((grid_x.ravel(),grid_y.ravel())))

lnd_y = interpolate.LinearNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,4])
grid_v = lnd_y(np.column_stack((grid_x.ravel(),grid_y.ravel())))

# merged everything back together to make single object
lnd_vel_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))


"""plotting the vectors of each of the averaged particle data"""
fig,axs = plt.subplots(2, 2,figsize=(14,14))
fig.suptitle("Scipy LinearND Interpolation Filament Velocity")
quiver = axs[0,0].quiver(vel_field[:,0], #x
                   vel_field[:,1], #y
                   vel_field[:,3], #u
                   vel_field[:,4], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,0].set_box_aspect(1)
axs[0,0].set_adjustable("datalim")
axs[0,0].set_title('Filament Velocity')
quiver = axs[0,1].quiver(lnd_vel_field[:,0], #x
                   lnd_vel_field[:,1], #y
                   lnd_vel_field[:,2], #u
                   lnd_vel_field[:,3], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,1].set_box_aspect(1)
axs[0,1].set_title('Interpolated Velocity')
axs[0,1].set_adjustable("datalim")

quiver = axs[1,0].quiver(vel_field[:,0], #x
                   vel_field[:,1], #y
                   vel_field[:,3], #u
                   vel_field[:,4], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,0].set_box_aspect(1)
axs[1,0].set_adjustable("datalim")
axs[1,0].set_xlim(100,200)
axs[1,0].set_ylim(100,200)
quiver = axs[1,1].quiver(lnd_vel_field[:,0], #x
                   lnd_vel_field[:,1], #y
                   lnd_vel_field[:,2], #u
                   lnd_vel_field[:,3], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,1].set_box_aspect(1)
axs[1,1].set_adjustable("datalim")
axs[1,1].set_xlim(100,200)
axs[1,1].set_ylim(100,200)
plt.show()

In [ ]:
max_x = np.max(vel_field[:,0])
min_x = np.min(vel_field[:,0])
max_y = np.max(vel_field[:,1])
min_y = np.min(vel_field[:,1])
print("X:",min_x,max_x,max_x-min_x)
print("Y:",min_y,max_y,max_y-min_y)

# create structured grid
interp_step = 1
grid_x,grid_y = np.meshgrid(np.arange(min_x,max_x,interp_step),
                           np.arange(min_y,max_y,interp_step))
num_interpolation_points = 75
grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                           np.linspace(min_y,max_y,num=num_interpolation_points))

# Interpolate the x and y components separately
lnd_x = interpolate.LinearNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,3])
grid_u = lnd_x(np.column_stack((grid_x.ravel(),grid_y.ravel())))

lnd_y = interpolate.LinearNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,4])
grid_v = lnd_y(np.column_stack((grid_x.ravel(),grid_y.ravel())))

# merged everything back together to make single object
lnd_vel_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))

vel_mag = np.sqrt(lnd_vel_field[:,2]**2 + lnd_vel_field[:,3]**2)
vorticity = np.gradient(np.reshape(lnd_vel_field[:,3],grid_x.shape), axis=0) - np.gradient(np.reshape(lnd_vel_field[:,2],grid_x.shape), axis=0)
print("vorticity",vorticity.shape)
"""plotting the vectors of each of the averaged particle data"""
fig,axs = plt.subplots(2, 2,figsize=(14,14))
fig.suptitle("LinearND Filament Velocity and Vorticity")
quiver = axs[0,0].quiver(vel_field[:,0], #x
                   vel_field[:,1], #y
                   vel_field[:,3], #u
                   vel_field[:,4], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,0].set_box_aspect(1)
axs[0,0].set_adjustable("datalim")
pcm = axs[0,0].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[0,0])
axs[0,0].set_title('Filament Velocity')

quiver = axs[0,1].quiver(lnd_vel_field[:,0], #x
                   lnd_vel_field[:,1], #y
                   lnd_vel_field[:,2], #u
                   lnd_vel_field[:,3], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,1].set_box_aspect(1)
axs[0,1].set_title('Interpolated Velocity')
axs[0,1].set_adjustable("datalim")
pcm = axs[0,1].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[0,1])

quiver = axs[1,0].quiver(vel_field[:,0], #x
                   vel_field[:,1], #y
                   vel_field[:,3], #u
                   vel_field[:,4], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,0].set_box_aspect(1)
axs[1,0].set_adjustable("datalim")
axs[1,0].set_xlim(100,200)
axs[1,0].set_ylim(100,200)
pcm = axs[1,0].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[1,0])

quiver = axs[1,1].quiver(lnd_vel_field[:,0], #x
                   lnd_vel_field[:,1], #y
                   lnd_vel_field[:,2], #u
                   lnd_vel_field[:,3], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,1].set_box_aspect(1)
axs[1,1].set_adjustable("datalim")
axs[1,1].set_xlim(100,200)
axs[1,1].set_ylim(100,200)
pcm = axs[1,1].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[1,1])
plt.show()

In [ ]:
from tqdm.auto import tqdm
print(nframes)
max_x = np.max(vel_field[:,0])
min_x = np.min(vel_field[:,0])
max_y = np.max(vel_field[:,1])
min_y = np.min(vel_field[:,1])
num_interpolation_points = 75
vorticity = np.zeros((nframes,num_interpolation_points,num_interpolation_points))
for iframe in tqdm(np.arange(nframes-500,nframes)):
   vel_field = get_particle_vel_frame(N,iframe,pos_data,vel_data,disp=False)
   
   grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                              np.linspace(min_y,max_y,num=num_interpolation_points))

   # Interpolate the x and y components separately
   lnd_x = interpolate.LinearNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,3])
   grid_u = lnd_x(np.column_stack((grid_x.ravel(),grid_y.ravel())))

   lnd_y = interpolate.LinearNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,4])
   grid_v = lnd_y(np.column_stack((grid_x.ravel(),grid_y.ravel())))

   # merged everything back together to make single object
   lnd_vel_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))
   # normalize the vector parts
   # n = np.sqrt(lnd_vel_field[:,2] ** 2 + lnd_vel_field[:,3] ** 2)
   # lnd_vel_field[:,2] = lnd_vel_field[:,2]/n
   # lnd_vel_field[:,3] = lnd_vel_field[:,3]/n

   vel_mag = np.sqrt(lnd_vel_field[:,2]**2 + lnd_vel_field[:,3]**2)
   vorticity[iframe,:,:] = np.gradient(np.reshape(lnd_vel_field[:,3],grid_x.shape), axis=0) - np.gradient(np.reshape(lnd_vel_field[:,2],grid_x.shape), axis=0)


In [ ]:
ta_vorticity = np.zeros((num_interpolation_points,num_interpolation_points))
for iframe in np.arange(nframes):
   ta_vorticity += vorticity[iframe,:,:]
   
ta_vorticity = ta_vorticity/nframes
fig,axs = plt.subplots(1, 1,figsize=(14,14))
fig.suptitle("Time-Averaged Vorticity")
pcm = axs.imshow(ta_vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs)
axs.set_title('Filament Velocity')

In [ ]:
max_x = np.max(vel_field[:,0])
min_x = np.min(vel_field[:,0])
max_y = np.max(vel_field[:,1])
min_y = np.min(vel_field[:,1])
print("X:",min_x,max_x,max_x-min_x)
print("Y:",min_y,max_y,max_y-min_y)

# create structured grid
interp_step = 1
grid_x,grid_y = np.meshgrid(np.arange(min_x,max_x,interp_step),
                           np.arange(min_y,max_y,interp_step))
num_interpolation_points = 75
grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                           np.linspace(min_y,max_y,num=num_interpolation_points))

# Interpolate the x and y components separately
nnd_x = interpolate.NearestNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,3])
grid_u = nnd_x(np.column_stack((grid_x.ravel(),grid_y.ravel())))

nnd_y = interpolate.NearestNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,4])
grid_v = nnd_y(np.column_stack((grid_x.ravel(),grid_y.ravel())))

# merged everything back together to make single object
nnd_vel_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))

"""plotting the vectors of each of the averaged particle data"""
fig,axs = plt.subplots(2, 2,figsize=(14,14))
fig.suptitle("Scipy LinearND Interpolation Filament Velocity")
quiver = axs[0,0].quiver(vel_field[:,0], #x
                   vel_field[:,1], #y
                   vel_field[:,3], #u
                   vel_field[:,4], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,0].set_box_aspect(1)
axs[0,0].set_adjustable("datalim")
axs[0,0].set_title('Filament Velocity')
quiver = axs[0,1].quiver(nnd_vel_field[:,0], #x
                   nnd_vel_field[:,1], #y
                   nnd_vel_field[:,2], #u
                   nnd_vel_field[:,3], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,1].set_box_aspect(1)
axs[0,1].set_title('Interpolated Velocity')
axs[0,1].set_adjustable("datalim")

quiver = axs[1,0].quiver(vel_field[:,0], #x
                   vel_field[:,1], #y
                   vel_field[:,3], #u
                   vel_field[:,4], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,0].set_box_aspect(1)
axs[1,0].set_adjustable("datalim")
axs[1,0].set_xlim(100,200)
axs[1,0].set_ylim(100,200)
quiver = axs[1,1].quiver(nnd_vel_field[:,0], #x
                   nnd_vel_field[:,1], #y
                   nnd_vel_field[:,2], #u
                   nnd_vel_field[:,3], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,1].set_box_aspect(1)
axs[1,1].set_adjustable("datalim")
axs[1,1].set_xlim(100,200)
axs[1,1].set_ylim(100,200)
plt.show()

In [ ]:
max_x = np.max(vel_field[:,0])
min_x = np.min(vel_field[:,0])
max_y = np.max(vel_field[:,1])
min_y = np.min(vel_field[:,1])
print("X:",min_x,max_x,max_x-min_x)
print("Y:",min_y,max_y,max_y-min_y)

# create structured grid
interp_step = 1
grid_x,grid_y = np.meshgrid(np.arange(min_x,max_x,interp_step),
                           np.arange(min_y,max_y,interp_step))
num_interpolation_points = 75
grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                           np.linspace(min_y,max_y,num=num_interpolation_points))

# Interpolate the x and y components separately
nnd_x = interpolate.NearestNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,3])
grid_u = nnd_x(np.column_stack((grid_x.ravel(),grid_y.ravel())))

nnd_y = interpolate.NearestNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,4])
grid_v = nnd_y(np.column_stack((grid_x.ravel(),grid_y.ravel())))

# merged everything back together to make single object
nnd_vel_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))

vel_mag = np.sqrt(nnd_vel_field[:,2]**2 + nnd_vel_field[:,3]**2)
vorticity = np.gradient(np.reshape(nnd_vel_field[:,3],grid_x.shape), axis=0) - np.gradient(np.reshape(nnd_vel_field[:,2],grid_x.shape), axis=0)
print("vorticity",vorticity.shape)
"""plotting the vectors of each of the averaged particle data"""
fig,axs = plt.subplots(2, 2,figsize=(14,14))
fig.suptitle("NearestND Filament Velocity and Vorticity")
quiver = axs[0,0].quiver(vel_field[:,0], #x
                   vel_field[:,1], #y
                   vel_field[:,3], #u
                   vel_field[:,4], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,0].set_box_aspect(1)
axs[0,0].set_adjustable("datalim")
pcm = axs[0,0].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[0,0])
axs[0,0].set_title('Filament Velocity')

quiver = axs[0,1].quiver(nnd_vel_field[:,0], #x
                   nnd_vel_field[:,1], #y
                   nnd_vel_field[:,2], #u
                   nnd_vel_field[:,3], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,1].set_box_aspect(1)
axs[0,1].set_title('Interpolated Velocity')
axs[0,1].set_adjustable("datalim")
pcm = axs[0,1].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[0,1])

quiver = axs[1,0].quiver(vel_field[:,0], #x
                   vel_field[:,1], #y
                   vel_field[:,3], #u
                   vel_field[:,4], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,0].set_box_aspect(1)
axs[1,0].set_adjustable("datalim")
axs[1,0].set_xlim(100,200)
axs[1,0].set_ylim(100,200)
pcm = axs[1,0].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[1,0])

quiver = axs[1,1].quiver(nnd_vel_field[:,0], #x
                   nnd_vel_field[:,1], #y
                   nnd_vel_field[:,2], #u
                   nnd_vel_field[:,3], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,1].set_box_aspect(1)
axs[1,1].set_adjustable("datalim")
axs[1,1].set_xlim(100,200)
axs[1,1].set_ylim(100,200)
pcm = axs[1,1].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[1,1])
plt.show()

In [ ]:
num_interpolation_points = 75
vorticity = np.zeros((nframes,num_interpolation_points,num_interpolation_points))
for iframe in tqdm(np.arange(nframes)):
   vel_field = get_particle_vel_frame(N,iframe,pos_data,vel_data,disp=False)
   
   grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                              np.linspace(min_y,max_y,num=num_interpolation_points))

   # Interpolate the x and y components separately
   nnd_x = interpolate.NearestNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,3])
   grid_u = nnd_x(np.column_stack((grid_x.ravel(),grid_y.ravel())))

   nnd_y = interpolate.NearestNDInterpolator(np.column_stack((vel_field[:,0],vel_field[:,1])),vel_field[:,4])
   grid_v = nnd_y(np.column_stack((grid_x.ravel(),grid_y.ravel())))

   # merged everything back together to make single object
   nnd_vel_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))
   # normalize the vector parts
   # n = np.sqrt(lnd_vel_field[:,2] ** 2 + lnd_vel_field[:,3] ** 2)
   # lnd_vel_field[:,2] = lnd_vel_field[:,2]/n
   # lnd_vel_field[:,3] = lnd_vel_field[:,3]/n

   vel_mag = np.sqrt(nnd_vel_field[:,2]**2 + nnd_vel_field[:,3]**2)
   vorticity[iframe,:,:] = np.gradient(np.reshape(nnd_vel_field[:,3],grid_x.shape), axis=0) - np.gradient(np.reshape(nnd_vel_field[:,2],grid_x.shape), axis=0)


In [ ]:
ta_vorticity = np.zeros((num_interpolation_points,num_interpolation_points))
for iframe in np.arange(nframes):
   ta_vorticity += vorticity[iframe,:,:]
   
ta_vorticity = ta_vorticity/nframes
fig,axs = plt.subplots(1, 1,figsize=(14,14))
fig.suptitle("Time-Averaged Vorticity")
pcm = axs.imshow(ta_vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs)
axs.set_title('Filament Velocity')

In [ ]:
max_x = np.max(vel_field[:,0])
min_x = np.min(vel_field[:,0])
max_y = np.max(vel_field[:,1])
min_y = np.min(vel_field[:,1])
print("X:",min_x,max_x,max_x-min_x)
print("Y:",min_y,max_y,max_y-min_y)

# create structured grid
interp_step = 1
grid_x,grid_y = np.meshgrid(np.arange(min_x,max_x,interp_step),
                           np.arange(min_y,max_y,interp_step))
num_interpolation_points = 100
grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                           np.linspace(min_y,max_y,num=num_interpolation_points))
# Interpolate the x and y components separately using bilinear interpolation
grid_u = interpolate.griddata((vector_field[:,0], vector_field[:,1]), vector_field[:,3], (grid_x, grid_y), method='linear')
grid_v = interpolate.griddata((vector_field[:,0], vector_field[:,1]), vector_field[:,4], (grid_x, grid_y), method='linear')
# merged everything back together to make single object
structured_vector_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))

vel_mag = np.sqrt(structured_vector_field[:,2]**2 + structured_vector_field[:,3]**2)
vorticity = np.gradient(np.reshape(structured_vector_field[:,3],grid_x.shape), axis=0) - np.gradient(np.reshape(structured_vector_field[:,2],grid_x.shape), axis=0)
print("vorticity",vorticity.shape)
"""plotting the vectors of each of the averaged particle data"""
fig,axs = plt.subplots(2, 2,figsize=(14,14))
fig.suptitle("Filament Velocity and Vorticity")
quiver = axs[0,0].quiver(vel_field[:,0], #x
                   vel_field[:,1], #y
                   vel_field[:,3], #u
                   vel_field[:,4], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,0].set_box_aspect(1)
axs[0,0].set_adjustable("datalim")
pcm = axs[0,0].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[0,0])
axs[0,0].set_title('Filament Velocity')

quiver = axs[0,1].quiver(structured_vector_field[:,0], #x
                   structured_vector_field[:,1], #y
                   structured_vector_field[:,2], #u
                   structured_vector_field[:,3], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[0,1].set_box_aspect(1)
axs[0,1].set_title('Interpolated Velocity')
axs[0,1].set_adjustable("datalim")
pcm = axs[0,1].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[0,1])

quiver = axs[1,0].quiver(vel_field[:,0], #x
                   vel_field[:,1], #y
                   vel_field[:,3], #u
                   vel_field[:,4], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,0].set_box_aspect(1)
axs[1,0].set_adjustable("datalim")
axs[1,0].set_xlim(100,200)
axs[1,0].set_ylim(100,200)
pcm = axs[1,0].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[1,0])

quiver = axs[1,1].quiver(structured_vector_field[:,0], #x
                   structured_vector_field[:,1], #y
                   structured_vector_field[:,2], #u
                   structured_vector_field[:,3], #v
                   pivot='mid',
                   color="tab:blue",
                   scale_units='xy',
                   scale=0.25
)
axs[1,1].set_box_aspect(1)
axs[1,1].set_adjustable("datalim")
axs[1,1].set_xlim(100,200)
axs[1,1].set_ylim(100,200)
pcm = axs[1,1].imshow(vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs[1,1])
plt.show()

In [ ]:
from tqdm.auto import tqdm
num_interpolation_points = 100
vorticity = np.zeros((nframes,num_interpolation_points,num_interpolation_points))
for iframe in tqdm(np.arange(nframes)):
   vel_field = get_particle_vel_frame(N,iframe,pos_data,vel_data,disp=False)
   
   grid_x,grid_y = np.meshgrid(np.linspace(min_x,max_x,num=num_interpolation_points),
                              np.linspace(min_y,max_y,num=num_interpolation_points))
   # Interpolate the x and y components separately using bilinear interpolation
   grid_u = interpolate.griddata((vector_field[:,0], vector_field[:,1]), vector_field[:,3], (grid_x, grid_y), method='linear')
   grid_v = interpolate.griddata((vector_field[:,0], vector_field[:,1]), vector_field[:,4], (grid_x, grid_y), method='linear')
   # merged everything back together to make single object
   structured_vector_field = np.column_stack((grid_x.flatten(), grid_y.flatten(), grid_u.flatten(), grid_v.flatten()))

   vel_mag = np.sqrt(structured_vector_field[:,2]**2 + structured_vector_field[:,3]**2)
   vorticity[iframe,:,:] = np.gradient(np.reshape(structured_vector_field[:,3],grid_x.shape), axis=0) - np.gradient(np.reshape(structured_vector_field[:,2],grid_x.shape), axis=0)

In [ ]:
ta_vorticity = np.zeros((num_interpolation_points,num_interpolation_points))
for iframe in np.arange(nframes):
   ta_vorticity += vorticity[iframe,:,:]
   
ta_vorticity = ta_vorticity/nframes
fig,axs = plt.subplots(1, 1,figsize=(14,14))
fig.suptitle("Time-Averaged Vorticity")
pcm = axs.imshow(ta_vorticity,cmap="coolwarm",alpha=0.5,origin="lower",extent=(min_x,max_x,min_y,max_y))
fig.colorbar(pcm, ax=axs)
axs.set_title('Filament Velocity')